In [1]:
import os
import pandas as pd
import numpy as np
import sys
import csv
from time import strptime
import json
from datetime import datetime as dt
from datetime import date
from statistics import mean
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

In [3]:
csv.field_size_limit(sys.maxsize)
np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_columns = None
pd.options.display.max_rows = 10000
pd.set_option('display.max_colwidth', -1)

/home/stas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [3]:
df = pd.read_csv('15 topics.csv')

In [4]:
list(df.columns)

['budget',
 'META__id',
 'PROCESS__original_language',
 'revenue',
 'runtime',
 'spoken_languages',
 'META__year',
 'META__month',
 'META__day',
 'weekend',
 'META__collection_name',
 'META__cast_1_name',
 'META__cast_2_name',
 'META__cast_3_name',
 'META__cast_4_name',
 'META__cast_5_name',
 'META__cast_6_name',
 'META__cast_7_name',
 'META__cast_8_name',
 'META__crew__sound__music_editor',
 'META__crew__sound__original_music_composer',
 'META__crew__sound__sound_designer',
 'META__crew__sound__sound_effects_editor',
 'META__crew__sound__sound_re_recording_mixer',
 'META__crew__sound__supervising_sound_editor',
 'META__crew__directing__director__1',
 'META__crew__directing__script_supervisor',
 'META__crew__production__casting',
 'META__crew__production__executive_producer__1',
 'META__crew__production__producer__1',
 'META__crew__production__producer__2',
 'META__crew__editing__editor__1',
 'META__crew__costume__costume_designer',
 'META__crew__costume__costume_supervisor',
 'META__c

In [8]:
def get_work_cols(df):
    return df[[col for col in df.columns if not 'META' in col and not 'PROCESS' in col]]

In [11]:
work_df = get_work_cols(df)
work_df.fillna(work_df.median(), inplace=True)

/home/stas/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [55]:
qzf = VarianceThreshold(threshold=0.01)
qzf.fit(work_df)

VarianceThreshold(threshold=0.01)

In [63]:
countries_to_other = [c for c in list(work_df.columns[[not t for t in qzf.get_support()]]) if 'country' in c]

In [67]:
len(countries_to_other)

63

In [271]:
def get_other_column_bool(df, columns_to_check):
    def is_other_column(row):
        for c in columns_to_check:
            if row[c]:
                return 1
        return 0
    
    column_other = []
    for i, row in df.iterrows():
        column_other.append(is_other_column(row))
    return column_other

In [177]:
df['country__other'] = get_other_column_bool(df, countries_to_other)

In [83]:
df_no_other_countries = df.drop(countries_to_other, axis=1)

In [84]:
list(df_no_other_countries.columns)

['budget',
 'META__id',
 'PROCESS__original_language',
 'revenue',
 'runtime',
 'spoken_languages',
 'META__year',
 'META__month',
 'META__day',
 'weekend',
 'META__collection_name',
 'META__cast_1_name',
 'META__cast_2_name',
 'META__cast_3_name',
 'META__cast_4_name',
 'META__cast_5_name',
 'META__cast_6_name',
 'META__cast_7_name',
 'META__cast_8_name',
 'META__crew__sound__music_editor',
 'META__crew__sound__original_music_composer',
 'META__crew__sound__sound_designer',
 'META__crew__sound__sound_effects_editor',
 'META__crew__sound__sound_re_recording_mixer',
 'META__crew__sound__supervising_sound_editor',
 'META__crew__directing__director__1',
 'META__crew__directing__script_supervisor',
 'META__crew__production__casting',
 'META__crew__production__executive_producer__1',
 'META__crew__production__producer__1',
 'META__crew__production__producer__2',
 'META__crew__editing__editor__1',
 'META__crew__costume__costume_designer',
 'META__crew__costume__costume_supervisor',
 'META__c

In [85]:
df_no_topics = df_no_other_countries.drop([c for c in list(df_no_other_countries.columns) if 'topic_' in c] , axis=1)

In [87]:
# get tmdb imdb ids
full_tmdb = pd.read_csv('data/tmdb/fulltmdb.csv')

In [89]:
tmdb_imdb_ids = full_tmdb[['id', 'imdb_id']]

In [102]:
ids = tmdb_imdb_ids[tmdb_imdb_ids.id.isin(df_no_topics.META__id)]

In [ ]:
def get_imdb_id(row):
    for imdb_i, imdb_row in ids.iterrows():
        if row['META__id'] == imdb_row['id']:
            return imdb_row['imdb_id']
    print("no_tmdb")
    return None

column_imdb = []
for i, row in df_no_topics.iterrows():
    if not i%100:
        print(i)
    column_imdb.append(get_imdb_id(row))

In [272]:
len(column_imdb)

7494

In [273]:
len([c for c in column_imdb if pd.isna(c)])

52

In [274]:
df_no_topics['META__imdb_id'] = column_imdb

In [275]:
with open('MPST/tag_assignment_data/movie_to_label_name.json') as json_file:
    movie_labels = json.load(json_file)

In [276]:
movie_labels

{'tt0055318': ['violence'],
 'tt0127919': ['paranormal'],
 'tt0272880': ['violence'],
 'tt0030442': ['comedy'],
 'tt0499168': ['romantic'],
 'tt0058648': ['anti war'],
 'tt0092534': ['romantic'],
 'tt1606610': ['violence', 'flashback'],
 'tt0077252': ['psychedelic', 'violence'],
 'tt0388125': ['dramatic', 'romantic', 'flashback'],
 'tt0364725': ['comedy', 'humor', 'entertaining', 'flashback'],
 'tt1369706': ['flashback'],
 'tt0057261': ['good versus evil', 'allegory', 'cult', 'satire', 'murder'],
 'tt0206275': ['violence', 'romantic', 'flashback'],
 'tt0299556': ['good versus evil'],
 'tt0112634': ['violence'],
 'tt0039428': ['flashback'],
 'tt0493402': ['murder'],
 'tt0203009': ['dramatic',
  'flashback',
  'psychedelic',
  'satire',
  'tragedy',
  'sentimental'],
 'tt0028231': ['suspenseful', 'murder'],
 'tt3110958': ['revenge', 'suspenseful', 'comedy', 'flashback'],
 'tt1483324': ['romantic'],
 'tt0384832': ['absurd', 'psychedelic', 'murder', 'violence', 'flashback'],
 'tt0085995': 

In [277]:
with open('MPST/tag_assignment_data/tag_list.txt') as f:
    content = f.readlines()
tag_list = [x.strip() for x in content]

In [ ]:
tag_columns = {}
for tag in tag_list:
    tag_columns[tag] = []
    
for i, row in df_no_topics.iterrows():
    if not i%100:
        print(i)
    movie_tags = movie_labels.get(row['META__imdb_id'], [])
    for tag in tag_list:
        tag_columns[tag].append(int(tag in movie_tags))

In [279]:
df_tags = df_no_topics.copy()
for tag in tag_columns:
    df_tags[f'tag__{tag}'.replace(' ', '_').replace('-', '_')] = tag_columns[tag]

In [280]:
df_only_tags = df_tags[[c for c in df_tags.columns if 'tag__' in c]]

qzf = VarianceThreshold(threshold=0.01)
qzf.fit(df_only_tags)

tags_to_other = [c for c in list(df_only_tags.columns[[not t for t in qzf.get_support()]])] 

In [281]:
list(set(list(df_only_tags.columns)) - set(tags_to_other))
# len(list(df_only_tags.columns)) - len(tags_to_other)

['tag__sentimental',
 'tag__prank',
 'tag__realism',
 'tag__entertaining',
 'tag__mystery',
 'tag__cruelty',
 'tag__plot_twist',
 'tag__historical',
 'tag__good_versus_evil',
 'tag__revenge',
 'tag__murder',
 'tag__inspiring',
 'tag__dramatic',
 'tag__flashback',
 'tag__alternate_reality',
 'tag__philosophical',
 'tag__absurd',
 'tag__paranormal',
 'tag__thought_provoking',
 'tag__humor',
 'tag__suspenseful',
 'tag__satire',
 'tag__neo_noir',
 'tag__horror',
 'tag__dark',
 'tag__sadist',
 'tag__gothic',
 'tag__tragedy',
 'tag__fantasy',
 'tag__action',
 'tag__melodrama',
 'tag__cute',
 'tag__bleak',
 'tag__comedy',
 'tag__psychological',
 'tag__romantic',
 'tag__psychedelic',
 'tag__depressing',
 'tag__stupid',
 'tag__storytelling',
 'tag__insanity',
 'tag__sci_fi',
 'tag__home_movie',
 'tag__atmospheric',
 'tag__cult',
 'tag__violence',
 'tag__boring']

In [283]:
df_tags_other = df_tags.copy()
df_tags_other['tag__other'] = get_other_column_bool(df_tags_other, tags_to_other)

In [284]:
df_tags_other.drop(tags_to_other, axis=1, inplace=True)

In [285]:
df_tags_other['META__imdb_id'] = df_tags['imdb_id']
df_tags_other.drop('imdb_id', axis=1, inplace=True)

In [286]:
list(df_tags_other.columns)

['budget',
 'META__id',
 'PROCESS__original_language',
 'revenue',
 'runtime',
 'spoken_languages',
 'META__year',
 'META__month',
 'META__day',
 'weekend',
 'META__collection_name',
 'META__cast_1_name',
 'META__cast_2_name',
 'META__cast_3_name',
 'META__cast_4_name',
 'META__cast_5_name',
 'META__cast_6_name',
 'META__cast_7_name',
 'META__cast_8_name',
 'META__crew__sound__music_editor',
 'META__crew__sound__original_music_composer',
 'META__crew__sound__sound_designer',
 'META__crew__sound__sound_effects_editor',
 'META__crew__sound__sound_re_recording_mixer',
 'META__crew__sound__supervising_sound_editor',
 'META__crew__directing__director__1',
 'META__crew__directing__script_supervisor',
 'META__crew__production__casting',
 'META__crew__production__executive_producer__1',
 'META__crew__production__producer__1',
 'META__crew__production__producer__2',
 'META__crew__editing__editor__1',
 'META__crew__costume__costume_designer',
 'META__crew__costume__costume_supervisor',
 'META__c

In [287]:
get_work_cols(df_tags_other).shape

(7494, 245)

In [288]:
df_tags_other[:20].META__cast_1_name

0     tim roth         
1     emilio estevez   
2     mark hamill      
3     albert brooks    
4     tom hanks        
5     kevin spacey     
6     orson welles     
7     bjrk             
8     bruce willis     
9     gustav frhlich   
10    sarah polley     
11    johnny depp      
12    uma thurman      
13    jamie foxx       
14    kieran o'brien   
15    marlon brando    
16    clint eastwood   
17    dan castellaneta 
18    jim carrey       
19    gael garca bernal
Name: META__cast_1_name, dtype: object

In [290]:
gender_cols = [c for c in list(df_tags_other.columns) if 'male' in c or 'female' in c]

In [4]:
import random
from nltk.corpus import names
import nltk
from nltk import NaiveBayesClassifier
from nltk.classify import apply_features

In [291]:

male_names = [(name, 'male') for name in names.words('male.txt')]
female_names = [(name, 'female') for name in names.words('female.txt')]
labeled_names = male_names + female_names
random.shuffle(labeled_names)

def gender_features(word):
    return {'last_letter': word[-1], 
            'last_two_letters': word[-2:]}

featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set = apply_features(gender_features, labeled_names)
# test_set = apply_features(gender_features, labeled_names[:500])
classifier = NaiveBayesClassifier.train(train_set)

In [292]:
gender_columns = {}
for i in range (1, 9):
    gender_columns[f'cast_{i}_gender'] = []

actors_predicted = []
for i, row in df_tags_other.iterrows():
    if not i%100:
        print(i)
    for j in range(1,9):
        name = row[f'META__cast_{j}_name']
        if pd.isna(name):
            gender = None
        elif not row[f'cast_{j}_male'] and not row[f'cast_{j}_female']:
            actors_predicted.append(name)
            gender = int(classifier.classify(gender_features(name.strip().split(' ')[0])) == 'male')
            print(f'{gender} - {name}')
        else:
            gender = row[f'cast_{j}_male']
        gender_columns[f'cast_{j}_gender'].append(gender)
print(f'classified actors: {len(actors_predicted)}')

0
1 - everlast
0 - vladica kostic
0 - maria de medeiros
1 - lucas black
1 - courtney taylor-taylor
1 - alex kapranos
1 - guy garvey
1 - peter hayes
1 - gruff rhys
1 - alvaro guerrero
1 - rodrigo murray
0 - heidi hayes
1 - daniel richter
1 - haymon maria buttinger
1 - tex rubinowitz
1 - liu zhong yuan
1 - zheng tia yong
1 - diabolo
1 - denis evrard
0 - yji matsuda
1 - yoshiko sakakibara
0 - iemasa kayumi
1 - mahito tsujimura
0 - christina zenato
1 - john charles
0 - cindy morgan
0 - antonia san juan
0 - toni cant
1 - michael badalucco
1 - brad sullivan
1 - michael scholes
0 - yji matsuda
0 - rumi hiiragi
0 - yumi tamai
1 - yasuko sawaguchi
1 - del pentecost
1 - francisco boira
1 - ral garca forneiro
1 - scott kolk
1 - solveig dommartin
1 - elmar wilms
1 - sihung lung
0 - fa zeng li
1 - su ying huang
1 - akiko takeshita
0 - kazuyoshi minamimagoe
1 - kazuko shibata
1 - paul lombardi
100
1 - james marcus
1 - carl duering
1 - james robinson
0 - amy robinson
0 - george memmoli
0 - fana mokoe

0 - gisela fischer
1 - simon chuckster
1 - melvin van peebles
1 - hubert scales
0 - lavelle roby
0 - rhetta hughes
1 - greg bensen
1 - ian cochrane
1 - brian h. dierker
1 - beau daniels
1 - jake milkovich
1 - ingrid rogers
1 - john savage
1 - scott moutter
0 - aida turturro
0 - camilla seberg
1 - paul roebling
1 - david sheiner
0 - kelly rowland
0 - veronica alicino
1 - jason narvy
1 - colin mccormack
1 - roger prncep
1 - montserrat carulla
1 - andrs gertrdix
1 - scar casas
0 - jessica alba
0 - megumi odaka
0 - kichi ueda
1 - leo meneghetti
1 - r. d. reid
1 - benjamin bratt
0 - seth allen
1 - john derek
1 - holland taylor
1 - antonio prester
1 - helen namaso lenamarken
1 - nicolas sironka
1 - damaris itenyo agweyu
1 - david gulpilil
0 - shea adams
1 - stephen jasso
1 - james bullard
1 - mike apaletegui
1 - adam chubbuck
1 - robert lujan
1 - boris leskin
1300
1 - irwin corey
1 - anjul nigam
1 - margot farley
1 - oren williams
1 - james remar
0 - emily siewert
0 - marissa matrone
1 - jas

1 - max beesley
1 - simon shepherd
1 - david benger
2300
1 - robert picardo
1 - daniel tay
1 - larry mccoy
1 - nigel hawthorne
1 - dehl berti
0 - steve mouzakis
1 - peter curtin
1 - phil hartman
0 - christina cole
0 - anthony arkin
0 - dina spybey-waters
1 - marc john jefferies
1 - andrew hampton
1 - jarred rumbold
1 - matthew price
1 - carl snell
0 - kopi sotiropulos
0 - elizabeth shepherd
0 - jamie wild
0 - steve rankin
1 - didier sauvegrain
0 - jeremy prevost
1 - jean-marc lentretien
1 - john dimaggio
0 - steve bickford
1 - patrick brady
1 - john dimaggio
0 - andy richter
0 - anne marie mcevoy
1 - robby kiger
0 - deborah rush
1 - stefanos miltsakakis
1 - mary wickliffe
0 - claudia stedelin
1 - ian nelson
1 - walton goggins
1 - aden gillett
1 - nicholas elliott
0 - magorzata foremniak
1 - wadysaw kowalski
0 - alicja sapryk
1 - michal breitenwald
1 - john voldstad
1 - chandler riley hecht
1 - david gane
1 - kent harper
1 - jason mcguire
1 - darrell hammond
1 - brian peck
1 - michael n

1 - toby hargrave
1 - yousuf azami
0 - kanaka
0 - kenny chin
1 - sanjay lafont
0 - amanda troya
0 - madelyn deutch
1 - calvin borel
0 - bruce wayne eckelman
1 - myriam charleins
1 - jonathan demurger
0 - ama quiambo
0 - gigi dueas
0 - laura centeno
1 - pen medina
0 - aya cash
0 - sonia deepti
0 - kamalinee mukherjee
1 - ryan corr
0 - philippe klaus
1 - srinivas
1 - saikumar pampana
0 - bhargavi
0 - ambati srinivas
1 - shankar rao
0 - madhumani
1 - eric allan kramer
1 - vhong navarro
0 - eugene domingo
1 - zaijan jaranilla
1 - aaron junatas
0 - basty alcances
1 - burn gorman
0 - sofia wells
1 - chad carlberg
1 - grant george
0 - shanelle workman
0 - cinda adams
0 - elisa gabrielli
1 - robin atkin downes
0 - sanaa alaoui
1 - yutian wang
0 - line pillet
0 - anushka sharma
1 - mithu chakrabarty
1 - dibyendu bhattacharya
0 - siddhanth kapoor
1 - ankur bhatia
1 - rajesh tailang
0 - daya shankar pandey
1 - samar jai singh
1 - sunil upadhyay
1 - nawazuddin siddiqui
0 - shraddha das
0 - divya d

0 - stephanie anderson
1 - chubby checker
1 - ohad knoller
0 - orly silbersatz banai
0 - ola schur selektar
1 - meir golan
0 - shlomi ben attar
1 - amir jerassi
1 - manoj bajpayee
1 - tigmanshu dhulia
1 - piyush mishra
1 - jameel khan
1 - ashwin mathew
1 - rajeev pahuja
1 - rambo rajkumar
1 - krishnakumar ramakumar
0 - masaya onosaka
0 - yuki kaida
1 - yasuhiro takato
1 - derek ting
1 - michael park
1 - darren e. scott
1 - jake boswell
0 - joshua milrad
1 - rod loomis
1 - chris phillips
1 - terry saunders
0 - pepita emmerichs
0 - steve peck
1 - russell buchanan
1 - j. kyle manzay
1 - raymond thiry
0 - jesse van driel
0 - joe pantoliano
1 - brian setzer
1 - harriet lenabe
0 - warona seane
0 - fiona ramsay
1 - alvin fitisemanu
1 - peter sa'ena brown
1 - cameron hopkin
1 - ineke den hollander
0 - jaime anderson
1 - andrew mcdonough
1 - roger imhof
1 - erin mcmurtry
0 - zie souwand
0 - sabyasachi chakraborty
1 - phyllis boyens
1 - foister dickerson
0 - lisa arrindell
1 - michael harding
0 

...

1 - christian contreras
1 - mark basnight
1 - john kaippallil
1 - efim petrunin
1 - sergey abroskin
0 - svetlana stepankovskaya
0 - chloe bennet
1 - albert tsai
0 - sarah paulson
0 - eye hadara
0 - lexie roth
0 - andria blackman
1 - david de beck
1 - aayam mehta
1 - ujjwal chopra
1 - al gore
1 - rakshit shetty
1 - pramod shetty
1 - dinesh mangalore
1 - jim pharr
1 - jordan beck
1 - jason ezzell
1 - rafe spall
0 - kerri mclean
1 - dash you
1 - poojan trivedi
0 - alisha prajapati
0 - netri trivedi
1 - prashant barot
1 - milind gunaji
1 - david manucharov
0 - ravi prakash
0 - lexie roth
1 - amrinder gill
1 - sardar sohi
0 - jayasudha
1 - yang yiwei
1 - mengke bateer
0 - anna chipovskaya
1 - aleksandr lykov
1 - yunus emre elik
1 - sezgin cengiz
1 - halit demir
1 - ilan dzbalan
1 - gastn pauls
1 - daniel roebuck
1 - james staub
1 - peyton lucas
1 - thomas staub
0 - valerie flores
1 - adrian flores
0 - svetlana alekseevna barandich
1 - eric r.gilliat
0 - anastasia vyazovskaya
0 - alina chorn

1 - kunaal roy kapur
1 - raju kher
1 - vijay raaz
1 - stephen dorff
1 - david wilmot
1 - meenal
1 - adukalam naren
0 - kofi ghanaba
1 - oyafunmike ogunlano
0 - alexandra duah
1 - nick medley
0 - mutabaruka
0 - mzuri
0 - elijah wood
1 - sharlto copley
1 - matt cross
0 - danny sapani
0 - issei ogata
1 - christopher denham
5500
0 - kenny wormald
1 - daniel aguirre
1 - luis bredow
0 - remy ishak
1 - tiz zaqyah
0 - mia sara nasuha
0 - liza othman
0 - syafie naswip
0 - suhasini maniratnam
0 - laila
1 - shyam ganesh
1 - sriman
1 - vijayan
1 - n. s. ramesh khanna
0 - laila
1 - vinod alva
1 - inigo prabhakaran
0 - srividhya
1 - karunas
1 - n. s. ramesh khanna
0 - vaiyapuri
0 - vindhya
1 - nizhalgal ravi
0 - raja
1 - karan
1 - manivannan
1 - rajeev
0 - chinni jayanth
1 - n. s. ramesh khanna
0 - radha bai
0 - laila
1 - ilavarasu
0 - paravai muniyamma
0 - ramya
1 - karunas
1 - rajeev khandelwal
1 - shiv pandit
1 - neil bhoopalam
1 - rajit kapoor
1 - tom berenger
1 - wolf muser
1 - serjo loroza
0 -

1 - john collin
0 - trisha todd
1 - karen trumbo
0 - faith mcdevitt
1 - craig damen
0 - leslie hidula
1 - caren graham
0 - sheila dickenson
0 - patricia blem
0 - birthe wolter
1 - daniel margotta
0 - dominique frot
0 - natasha mashkevich
1 - jan rippe
0 - laila westersund
0 - charlotte strandberg
1 - bert wheeler
1 - harry o'toole
1 - wheeler oakman
0 - minta durfee
0 - minnie devereaux
0 - priyanka xi
1 - jeff truman
1 - karthik muthuraman
0 - munna simon
0 - vaiyapuri
0 - ksenia nikolaeva
0 - yekaterina vulichenko
1 - michael welz
0 - nieve de medina
1 - jos ngel egido
1 - mary passeri
1 - michael higgenbottom
0 - blaine capatch
0 - amy stewart
1 - xing yu
1 - tom wood
1 - cosmo
0 - omi vaidya
1 - shazahn padamsee
0 - shruti haasan
1 - dj flava
1 - aj diana
1 - jos julin
1 - carlos linares
1 - william morse
0 - blessy
0 - tulasi
1 - denis nikiforov
0 - dmitriy shevchenko
0 - karisma kapoor
1 - bikram saluja
0 - asha sachdev
0 - bridgette wilson
1 - armin shimerman
1 - fran perea
1 - 

0 - keri hilson
1 - hillary baack
1 - denzel washington
0 - joe massingill
1 - ayaan khan
1 - william jhnk nielsen
0 - paola borboni
0 - valeri zavodovsky
0 - sergei volzhin-yastrebov
0 - julia govor
1 - jean michel par
1 - ian helfer
1 - frank aragon
1 - james carreras
0 - frankie avina
0 - steve nagribianko
1 - glen thompson
0 - samantha king
0 - barbara brenner
0 - barbara ehrenreich
0 - nancy brinker
1 - susan love
1 - david artus
1 - kevin furlong
1 - suman ranganathan
1 - laxmikant berde
1 - fallon goodson
1 - john wu
0 - katarina choi
1 - bud abbott
0 - beverly arlynne
1 - al baker jr.
1 - nat bodian
1 - eric sheffer stevens
1 - adam barnett
0 - eli arlis
0 - donna biscoe
0 - brady coleman
0 - tommy g. kendrick
0 - dale dudley
0 - tuba nsal
1 - nejat uygur
6800
0 - mamta kulkarni
1 - max renaudin
1 - mohamed fellag
1 - ray sahetapy
0 - pierre gruno
0 - ananda george
1 - tegar satrya
1 - igor zaripov
1 - matt gillanders
1 - jason berrent
0 - sophia elisabeth
1 - lutz halbhubner
1

1 - sergio avelar
1 - michael aguero
7200
1 - scott eastwood
1 - aleksandr golovin
1 - ivan dorn
0 - ekaterina guseva
0 - reese alexander
0 - keene mcrae
0 - suzanne reuter
0 - nora mcmenamy
0 - vijayendra ghatge
0 - joe massingill
1 - sam lee
0 - maria bello
1 - ajmal ameer
1 - andrew brooke
1 - brandon gonzales
1 - cameron mills
0 - savika chaiyadej
0 - nerea barros
1 - jess castro
1 - manolo solo
0 - cecilia villanueva
1 - connor corum
0 - lane styles
0 - resse hartwig
1 - cassius willis
1 - matthew goode
1 - rachel marie lewis
1 - jordan rhodes
1 - rob neukirch
1 - bo keister
1 - blu de golyer
1 - jill scott
1 - roy hudd
0 - breanne hill
1 - charles aitken
1 - tom morrissey
1 - paul reubens
1 - harold livingston
1 - jake gyllenhaal
1 - zac efron
1 - matthew mcconaughey
0 - jayne middlemiss
1 - ryan spielman
0 - anushka sharma
1 - kay kay menon
0 - siddhartha basu
1 - remo fernandes
0 - madina nalwanga
1 - martin kabanza
0 - taryn kay kyaze
1 - esther tebandeke
1 - ethan nazario lub

In [243]:
len(set(actors_predicted))

6207

In [293]:
df_gender = df_tags_other.copy()
for col in list(gender_columns.keys()):
    df_gender[col] = gender_columns[col]
    
for i in range(1,9):
    df_gender[f'META__cast_{i}_male'] = df_gender[f'cast_{i}_male']
    df_gender[f'META__cast_{i}_female'] = df_gender[f'cast_{i}_female']
    df_gender.drop([f'cast_{i}_male', f'cast_{i}_female'], axis=1, inplace=True)

In [294]:
list(df_gender.columns)

['budget',
 'META__id',
 'PROCESS__original_language',
 'revenue',
 'runtime',
 'spoken_languages',
 'META__year',
 'META__month',
 'META__day',
 'weekend',
 'META__collection_name',
 'META__cast_1_name',
 'META__cast_2_name',
 'META__cast_3_name',
 'META__cast_4_name',
 'META__cast_5_name',
 'META__cast_6_name',
 'META__cast_7_name',
 'META__cast_8_name',
 'META__crew__sound__music_editor',
 'META__crew__sound__original_music_composer',
 'META__crew__sound__sound_designer',
 'META__crew__sound__sound_effects_editor',
 'META__crew__sound__sound_re_recording_mixer',
 'META__crew__sound__supervising_sound_editor',
 'META__crew__directing__director__1',
 'META__crew__directing__script_supervisor',
 'META__crew__production__casting',
 'META__crew__production__executive_producer__1',
 'META__crew__production__producer__1',
 'META__crew__production__producer__2',
 'META__crew__editing__editor__1',
 'META__crew__costume__costume_designer',
 'META__crew__costume__costume_supervisor',
 'META__c

In [295]:
df_gender['META__revenue'] = df_gender['revenue']
df_gender.drop('revenue', axis=1, inplace=True)

In [296]:
df_gender.to_csv('16 df_gender.csv', mode='w+', index=False)